### Introduction
Ce notebook a pour objectif de charger et de préparer les données nécessaires pour l'optimisation des ateliers en fin d'année scolaire. Les données proviennent d'un fichier Excel nommé "Template.xlsx", qui contient deux onglets :

- **Ateliers** : Informations sur les ateliers disponibles.
- **Preferences** : Préférences des élèves pour les ateliers.

### Étape 1 : Importer les bibliothèques nécessaires
Nous allons utiliser la bibliothèque pandas pour manipuler les données.

In [14]:
# Importer la bibliothèque pandas
import pandas as pd

### Étape 2 : Charger les données depuis le fichier Excel
Nous allons lire les deux onglets du fichier Excel et stocker les données dans des DataFrames.

In [15]:
# Définir le nom du fichier Excel
nom_fichier = 'Template.xlsx'

# Charger les données de l'onglet 'Ateliers'
df_ateliers = pd.read_excel(nom_fichier, sheet_name='Ateliers')

# Charger les données de l'onglet 'Preferences'
df_preferences = pd.read_excel(nom_fichier, sheet_name='Preferences')

### Étape 3 : Afficher un aperçu des données chargées et préparation des données
Il est important de vérifier que les données ont été correctement chargées.

In [16]:
# Afficher les premières lignes du DataFrame des ateliers
print("Aperçu des Ateliers :")
display(df_ateliers.head())

# Afficher les premières lignes du DataFrame des préférences
print("Aperçu des Préférences :")
display(df_preferences.head())

Aperçu des Ateliers :


,Code,Description,Enseignant,Nombre de périodes,Nombre d'élèves max par session,Nombre idéal d'élèves par session,Lundi matin,Lundi après-midi,Mardi matin,Mardi après-midi,Mercredi
0,E01,Visite musée A,Prof 1,4,15,14,1.0,NaN,NaN,NaN,NaN
1,E02,Visite musée B,Prof 2,4,30,25,NaN,1.0,1.0,NaN,NaN
2,E03,Visite musée C,Prof 3,4,15,14,NaN,NaN,NaN,NaN,1.0
3,E04,Visite musée D,Prof 4,4,30,25,NaN,NaN,NaN,1.0,NaN
4,J01,Jass,Prof 5,4,15,14,1.0,NaN,1.0,NaN,NaN


Aperçu des Préférences :


,Nom,Prénom,Classe,# Préférences,E01,E02,E03,E04,J01,J02,...,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10
0,Dubois,Emma,10VG/1,13,1.0,NaN,1.0,NaN,1.0,NaN,...,1.0,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0
1,Martinet,Noah,10VG/1,13,1.0,NaN,1.0,-1.0,1.0,NaN,...,NaN,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0
2,Favre,Mia,10VG/1,13,1.0,NaN,1.0,NaN,1.0,-1.0,...,NaN,1.0,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0
3,Roux,Gabriel,10VG/1,13,1.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0
4,Girard,Chloe,10VG/1,13,NaN,1.0,1.0,NaN,-1.0,1.0,...,1.0,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0


#### 3.1 Transformation des préférences des élèves
Nous allons transformer le DataFrame des préférences pour faciliter l'assignation des ateliers.

In [17]:
# Remplacer les valeurs NaN par 0 dans les colonnes des ateliers
colonnes_ateliers = df_preferences.columns[4:]  # Les colonnes des ateliers commencent à l'indice 4
df_preferences[colonnes_ateliers] = df_preferences[colonnes_ateliers].fillna(0)

# Convertir les valeurs en entiers
df_preferences[colonnes_ateliers] = df_preferences[colonnes_ateliers].astype(int)

# Afficher le DataFrame des préférences mis à jour
print("Préférences après transformation :")
display(df_preferences.head())

Préférences après transformation :


,Nom,Prénom,Classe,# Préférences,E01,E02,E03,E04,J01,J02,...,A01,A02,A03,A04,A05,A06,A07,A08,A09,A10
0,Dubois,Emma,10VG/1,13,1,0,1,0,1,0,...,1,0,0,1,1,1,0,1,1,1
1,Martinet,Noah,10VG/1,13,1,0,1,-1,1,0,...,0,0,1,1,1,1,0,1,1,1
2,Favre,Mia,10VG/1,13,1,0,1,0,1,-1,...,0,1,0,1,1,1,0,1,1,1
3,Roux,Gabriel,10VG/1,13,1,0,1,0,1,0,...,0,0,0,1,1,1,0,1,1,1
4,Girard,Chloe,10VG/1,13,0,1,1,0,-1,1,...,1,0,0,1,1,1,0,1,1,1


#### 3.2 Traitement des disponibilités des ateliers
Nous allons créer une structure qui indique pour chaque atelier les sessions disponibles.

In [18]:
# Liste des sessions
sessions = ['Lundi matin', 'Lundi après-midi', 'Mardi matin', 'Mardi après-midi', 'Mercredi']

# Créer une colonne 'Sessions disponibles' pour chaque atelier
def obtenir_sessions_disponibles(row):
    disponibles = []
    for session in sessions:
        if row[session] == 1:
            disponibles.append(session)
    return disponibles

df_ateliers['Sessions disponibles'] = df_ateliers.apply(obtenir_sessions_disponibles, axis=1)

# Supprimer les colonnes individuelles des sessions (facultatif)
df_ateliers = df_ateliers.drop(columns=sessions)

# Afficher le DataFrame des ateliers mis à jour
print("Ateliers après traitement des sessions disponibles :")
display(df_ateliers.head())


Ateliers après traitement des sessions disponibles :


,Code,Description,Enseignant,Nombre de périodes,Nombre d'élèves max par session,Nombre idéal d'élèves par session,Sessions disponibles
0,E01,Visite musée A,Prof 1,4,15,14,[Lundi matin]
1,E02,Visite musée B,Prof 2,4,30,25,"[Lundi après-midi, Mardi matin]"
2,E03,Visite musée C,Prof 3,4,15,14,[Mercredi]
3,E04,Visite musée D,Prof 4,4,30,25,[Mardi après-midi]
4,J01,Jass,Prof 5,4,15,14,"[Lundi matin, Mardi matin]"


### Étape 4 : Analyse des données et vérification des contraintes
Dans cette étape, nous allons :

Identifier les élèves qui n'ont pas exactement 13 préférences.
Identifier les élèves qui ont plus de 2 veto (-1).
Identifier les ateliers qui n'ont aucune session disponible.

#### 4.1 Élèves avec un nombre incorrect de préférences

In [19]:
# Calculer le nombre de préférences (valeurs 1) par élève
df_preferences['Nombre de préférences'] = (df_preferences[colonnes_ateliers] == 1).sum(axis=1)

# Identifier les élèves qui n'ont pas exactement 13 préférences
eleves_pref_incorrectes = df_preferences[df_preferences['Nombre de préférences'] != 13]

# Afficher les élèves concernés
print("Élèves avec un nombre de préférences différent de 13 :")
display(eleves_pref_incorrectes[['Nom', 'Prénom', 'Nombre de préférences']])

Élèves avec un nombre de préférences différent de 13 :


,Nom,Prénom,Nombre de préférences
18,Chatelain,Camille,0
19,Schmid,Nathan,0
20,Maillard,Charlotte,0
21,Michel,Paul,0
22,Jordan,Eva,0
23,Aubry,Martin,0
24,Perrin,Leonie,0
25,Petit,Etienne,0
26,Bonnard,Maude,0
27,Monnier,Alexandre,0


#### 4.2 Élèves avec plus de 2 veto (-1)

In [20]:
# Calculer le nombre de veto (valeurs -1) par élève
df_preferences['Nombre de veto'] = (df_preferences[colonnes_ateliers] == -1).sum(axis=1)

# Identifier les élèves qui ont plus de 2 veto
eleves_trop_veto = df_preferences[df_preferences['Nombre de veto'] > 2]

# Afficher les élèves concernés
print("Élèves avec plus de 2 veto :")
display(eleves_trop_veto[['Nom', 'Prénom', 'Nombre de veto']])

Élèves avec plus de 2 veto :


,Nom,Prénom,Nombre de veto
16,Bovet,Clara,3


#### 4.3 Ateliers sans session disponible

In [21]:
# Vérifier les ateliers sans session disponible
ateliers_sans_session = df_ateliers[df_ateliers['Sessions disponibles'].apply(len) == 0]

# Afficher les ateliers concernés
print("Ateliers sans session disponible :")
display(ateliers_sans_session[['Code', 'Description']])

Ateliers sans session disponible :


,Code,Description
20,A09,Création d'œuvres à partir de matériaux recyclés


### Étape 5 : Statistiques sur les préférences et les veto

#### 5.1 Nombre moyen de préférences par activité


In [22]:
# Calculer le nombre de préférences (valeurs 1) pour chaque atelier
preferences_par_atelier = (df_preferences[colonnes_ateliers] == 1).sum()

# Calculer le nombre moyen de préférences
nombre_moyen_preferences = preferences_par_atelier.mean()

print(f"Nombre moyen de préférences par activité : {nombre_moyen_preferences:.2f}")

Nombre moyen de préférences par activité : 10.64


5.2 Top 3 des activités les plus préférées

In [23]:
# Obtenir les 3 activités avec le plus de préférences
top3_activites = preferences_par_atelier.sort_values(ascending=False).head(3)

print("Top 3 des activités les plus préférées :")
display(top3_activites)


Top 3 des activités les plus préférées :


A10    18
E03    18
A09    18
dtype: int64

5.3 Top 3 des activités les moins préférées

In [24]:
# Obtenir les 3 activités avec le moins de préférences
bottom3_activites = preferences_par_atelier.sort_values().head(3)

print("Top 3 des activités les moins préférées :")
display(bottom3_activites)

Top 3 des activités les moins préférées :


A07    0
E04    0
J04    0
dtype: int64

#### 5.4 Activités avec le plus de veto

In [25]:
# Calculer le nombre de veto (valeurs -1) pour chaque atelier
veto_par_atelier = (df_preferences[colonnes_ateliers] == -1).sum()

# Obtenir les activités avec le plus de veto
activites_plus_veto = veto_par_atelier.sort_values(ascending=False).head(3)

print("Activités avec le plus de veto :")
display(activites_plus_veto)


Activités avec le plus de veto :


J04    3
A02    3
E04    2
dtype: int64

### Étape 6 : Préparation des données pour l'optimisation
Nous allons préparer les structures de données nécessaires pour l'algorithme d'optimisation.

In [26]:
# Créer une liste des élèves
liste_eleves = df_preferences['Nom'] + ' ' + df_preferences['Prénom']

# Créer une liste des codes des ateliers
liste_ateliers = colonnes_ateliers

# Créer un dictionnaire des préférences
# Clé : Élève, Valeur : Dictionnaire des préférences pour chaque atelier
preferences_eleves = {}
for index, row in df_preferences.iterrows():
    eleve = row['Nom'] + ' ' + row['Prénom']
    preferences = row[colonnes_ateliers].to_dict()
    preferences_eleves[eleve] = preferences

# Afficher un exemple des préférences d'un élève
print("Préférences de l'élève {} :".format(liste_eleves.iloc[0]))
print(preferences_eleves[liste_eleves.iloc[0]])


Préférences de l'élève Dubois Emma :
{'E01': 1, 'E02': 0, 'E03': 1, 'E04': 0, 'J01': 1, 'J02': 0, 'J03': 0, 'J04': -1, 'J05': 0, 'J06': 1, 'J07': 1, 'J08': 1, 'A01': 1, 'A02': 0, 'A03': 0, 'A04': 1, 'A05': 1, 'A06': 1, 'A07': 0, 'A08': 1, 'A09': 1, 'A10': 1}
